<a href="https://colab.research.google.com/github/v670/MLProjects/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_train=pd.read_csv('/content/drive/MyDrive/datasets/yelp_review_polarity_csv/train.csv')

In [ ]:
df_train.columns = ['Ratings','Comments']

In [ ]:
df=df_train[:10000]

In [ ]:
df.head()

,Ratings,Comments
0,2,Been going to Dr. Goldberg for over 10 years. ...
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,2,All the food is great here. But the best thing...
4,1,Wing sauce is like water. Pretty much a lot of...


In [ ]:
df.shape

(10000, 2)

In [ ]:
X=list(df['Comments'])
y=list(df['Ratings'])

In [ ]:
y=list(pd.get_dummies(y,drop_first=True)[2])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
len(X_train)

8000

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')



In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
train_tf_dataset = train_dataset.shuffle(len(train_dataset)).batch(16)


In [ ]:
from transformers import TFDistilBertForSequenceClassification

bertmodel=TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5)
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
bertmodel.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

bertmodel.fit(train_tf_dataset,epochs=2,batch_size=16)


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
model.evaluate(test_dataset.shuffle(500).batch(16),return_dict=True,batch_size=16)

125/125 [==============================] - 77s 598ms/step - loss: 0.1779 - accuracy: 0.9355


{'accuracy': 0.9355000257492065, 'loss': 0.17789004743099213}

In [ ]:
model.predict(test_dataset)

TFSequenceClassifierOutput([('logits', array([[ 0.23535201, -0.09485961],
                                    [-0.85921985,  1.1174635 ],
                                    [-1.114382  ,  1.1546471 ],
                                    ...,
                                    [ 0.01680257,  0.11754263],
                                    [ 0.01680257,  0.11754263],
                                    [ 0.01680257,  0.11754263]], dtype=float32))])

In [ ]:
outputs=model.predict(test_dataset)[0]

In [ ]:
label=tf.argmax(outputs,axis=1)

In [ ]:
label.numpy()

array([0, 1, 1, ..., 1, 1, 1])

In [ ]:
model.save('/content/drive/MyDrive/sentiment_analysis') 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/sentiment_analysis/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/sentiment_analysis/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model('/content/drive/MyDrive/sentiment_analysis')

In [ ]:
import tempfile
import os

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    new_model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1



INFO:tensorflow:Assets written to: /tmp/1/assets


INFO:tensorflow:Assets written to: /tmp/1/assets



Saved model:
total 3652
drwxr-xr-x 2 root root    4096 Sep 27 08:10 assets
-rw-r--r-- 1 root root   68545 Sep 27 08:10 keras_metadata.pb
-rw-r--r-- 1 root root 3658894 Sep 27 08:10 saved_model.pb
drwxr-xr-x 2 root root    4096 Sep 27 08:10 variables


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
MODEL_DIR

'/tmp'

In [ ]:
!saved_model_cli show --dir {MODEL_DIR}/1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_ids/input_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 5)
      name: serving_default_input_ids/input_ids:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['logits'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [ ]:
MODEL_DIR


'/tmp'

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  35890      0 --:--:-- --:--:-- --:--:-- 35890
OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/

In [ ]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 79 not upgraded.
Need to get 347 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.6.0 [347 MB]
Fetched 347 MB in 5s (68.9 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.6.0_all.deb ...
Unpacking tensorflow-model-server (2.6.0) ...
Setting up tensorflow-model-server (2.6.0) ...


In [ ]:
!pip install -q requests

In [ ]:
import requests

In [ ]:
import os
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8509 \
  --model_name=sentiment_analysis \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [ ]:
!tail server.log

2021-09-27 08:11:23.398440: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:283] SavedModel load for tags { serve }; Status: success: OK. Took 662659 microseconds.
2021-09-27 08:11:23.417309: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2021-09-27 08:11:23.417488: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: sentiment_analysis version: 1}
2021-09-27 08:11:23.419652: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-09-27 08:11:23.419745: I tensorflow_serving/model_servers/server.cc:133] Using InsecureServerCredentials
2021-09-27 08:11:23.419760: I tensorflow_serving/model_servers/server.cc:383] Profiler service is enabled
2021-09-27 08:11:23.420272: I tensorflow_serving/model_servers/server.cc:409] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] getaddrinfo: address family for n

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
sentence = "I hate this movie"
batch=tokenizer(sentence)
batch=dict(batch)
batch=[batch]

In [ ]:
batch[0]

{'attention_mask': [1, 1, 1, 1, 1, 1],
 'input_ids': [101, 1045, 5223, 2023, 3185, 102]}